# **Arabic Text Summarization using AraBART**

In [ ]:
!pip install transformers sentencepiece torch arabert datasets accelerate rouge

In [ ]:

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig
from sklearn.model_selection import train_test_split
import numpy as np
from dataclasses import dataclass
from transformers import AutoTokenizer
from torch.utils.data import Dataset
from transformers import AutoConfig, AutoModelForSeq2SeqLM
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn


In [ ]:
from datasets import load_dataset
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import pandas as pd


# 1. Dataset Download and Loading

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mostafafawzymamoud","key":"a7f1a657edd6da516ac6814cb0abf3d7"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download the dataset
!kaggle datasets download -d abdalrahmanshahrour/arabicsummarization

# Unzip
!unzip -o arabicsummarization.zip -d arabicsummarization

import os

# List all files in the folder
for root, dirs, files in os.walk("arabicsummarization"):
    for file in files:
        print(file)

Dataset URL: https://www.kaggle.com/datasets/abdalrahmanshahrour/arabicsummarization
License(s): unknown
Archive:  arabicsummarization.zip
  inflating: arabicsummarization/data.csv  
  inflating: arabicsummarization/summarizdataset.csv  
data.csv
summarizdataset.csv


In [ ]:

df = pd.read_csv("arabicsummarization/summarizdataset.csv")
df.shape

(8378, 4)

In [ ]:
data=df
data.head()

,text,type,Processed Text,summarizer
0,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...,culture,اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم ب...,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...
1,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون...",culture,تحصل كتاب المصحف وقراءاته الفه باحثون تونسيون ...,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون..."
2,\nاحتضن جناح تونس في القرية الدولية للأفلام بم...,culture,احتضن جناح تونس القريه الدوليه للافلام بمدينه ...,تونس حاضرة من جهة أخرى ستكون تونس حاضرة في قائ...
3,\nشهدت برلين أمس الجمعة افتتاح مسجد فريد من نو...,culture,شهدت برلين الجمعه افتتاح مسجد فريد نوعه الاقل ...,واستأجرت صاحبة المشروع المحامية والكاتبة سيران...
4,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...,culture,نعت وزاره المنشد عز بن محمود انتقل جوار يوم تن...,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...


# 2. Data Cleaning and Preprocessing

 Drop Irrelevant Columns

---




In [ ]:
data.drop('type',inplace=True,axis=1)
data.drop('text',inplace=True,axis=1)
data.head()

,Processed Text,summarizer
0,اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم ب...,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...
1,تحصل كتاب المصحف وقراءاته الفه باحثون تونسيون ...,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون..."
2,احتضن جناح تونس القريه الدوليه للافلام بمدينه ...,تونس حاضرة من جهة أخرى ستكون تونس حاضرة في قائ...
3,شهدت برلين الجمعه افتتاح مسجد فريد نوعه الاقل ...,واستأجرت صاحبة المشروع المحامية والكاتبة سيران...
4,نعت وزاره المنشد عز بن محمود انتقل جوار يوم تن...,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...


Clean and Preprocess Text

---



In [ ]:
#Use NLTK Arabic stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('arabic'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import re
def clean_text(text):
    text = re.sub(r'[^\u0621-\u064A\u0660-\u0669\s0-9():.،؛]+', '', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub(r'\.{2,}', '', text)
    return text.strip()

In [ ]:
def preprocess_text(text):
    text = clean_text(text)
    tokens = text.split()
    filtered = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered)


In [ ]:
data['Processed Text'] = data['Processed Text'].apply(preprocess_text)
print(data['Processed Text'])

0       اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم ب...
1       تحصل كتاب المصحف وقراءاته الفه باحثون تونسيون ...
2       احتضن جناح تونس القريه الدوليه للافلام بمدينه ...
3       شهدت برلين الجمعه افتتاح مسجد فريد نوعه الاقل ...
4       نعت وزاره المنشد عز بن محمود انتقل جوار يوم تن...
                              ...                        
8373    تاجل الاضراب العام قطاع الصحه مقررا تنفيذه الي...
8374    كشف الناشطان كريم نوار وعفيف زقيه اشرفا عمليه ...
8375    فرقه الابحاث والتفتيش للحرس الوطني بطبلبه ولاي...
8376    قرر الاهالي بمناطق هيشر وعين القارصي والغولايث...
8377    تمكنت وحدات الحرس الوطني بمحطه الاستخلاص ببرج ...
Name: Processed Text, Length: 8378, dtype: object


# 3. Split Dataset into Train / Validation / Test

In [ ]:
# Split data into train, validation, and test sets
def split_data(df, train_size=0.8, val_size=0.1):
    train_df, temp_df = train_test_split(df, train_size=train_size, random_state=42)
    val_df, test_df = train_test_split(temp_df, train_size=val_size/(1-train_size), random_state=42)
    return train_df, val_df, test_df

train_df, val_df, test_df = split_data(data)


# 4. Dataset Wrapping using PyTorch Dataset

Defines a custom PyTorch Dataset to handle our Arabic summarization dataset. This includes:

*   A LabeledExample dataclass to structure each (paragraph, summary) pair.
*   A LabeledDataset class that:

    *   Tokenizes both input texts (paragraphs) and target texts (summaries).
    *   Returns a dictionary compatible with training loops.






In [ ]:
from dataclasses import dataclass
from torch.utils.data import Dataset
from transformers import AutoTokenizer

@dataclass
class LabeledExample:
    paragraph: str
    summary: str

class LabeledDataset(Dataset):
    def __init__(self, data: list[LabeledExample], tokenizer_name='moussaKam/AraBART'):
        self.data = data
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, idx):
        # Access paragraph and summary from the LabeledExample object
        example = self.data[idx]
        paragraph = example.paragraph
        summary = example.summary

        inputs = self.tokenizer.encode_plus(
            paragraph,
            truncation=True,
            padding='max_length',
            max_length=512,
            return_tensors='pt'
        )
        labels = self.tokenizer.encode_plus(
            summary,
            truncation=True,
            padding='max_length',
            max_length=110,
            return_tensors='pt'
        )

        # Return a dictionary including the summarizer
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': labels['input_ids'].flatten(),
            'summarizer': summary,
            'paragraph': paragraph,
            'example_id': str(idx)
        }


In [ ]:
# Convert train, validation, and test data to LabeledExample format
def load_labeled_dataset(df):
    dataset = []
    for _, row in df.iterrows():
        paragraph = row['Processed Text']
        summary = row['summarizer']
        dataset.append(LabeledExample(paragraph=paragraph, summary=summary))  # Use LabeledExample
    return dataset

train_dataset = LabeledDataset(load_labeled_dataset(train_df))
val_dataset = LabeledDataset(load_labeled_dataset(val_df))
test_dataset = LabeledDataset(load_labeled_dataset(test_df))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

# 5. Initialize and Prepare the AraBART Model

In [ ]:
# Initialize the model
def initialize_model(pretrained_model_name, device):
    model_config = AutoConfig.from_pretrained(pretrained_model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name, config=model_config)
    model.to(device)
    return model

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = initialize_model('moussaKam/AraBART', device)


pytorch_model.bin:   0%|          | 0.00/557M [00:00<?, ?B/s]

# 📚6. Tokenizer Setup and DataLoaders

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("moussaKam/AraBART")
tokenizer.save_pretrained("/content/trained_model")

model.safetensors:   0%|          | 0.00/557M [00:00<?, ?B/s]

('/content/trained_model/tokenizer_config.json',
 '/content/trained_model/special_tokens_map.json',
 '/content/trained_model/sentencepiece.bpe.model',
 '/content/trained_model/added_tokens.json',
 '/content/trained_model/tokenizer.json')

In [ ]:
def create_data_loader(dataset, batch_size):
    return DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

batch_size = 8
train_loader = create_data_loader(train_dataset, batch_size)
val_loader = create_data_loader(val_dataset, batch_size)
test_loader = create_data_loader(test_dataset, batch_size)


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


# 7. Training Configuration and Loop

In [ ]:
# Initialize training components
def initialize_training_components(model, learning_rate):
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    return optimizer, criterion

learning_rate = 5e-5
optimizer, criterion = initialize_training_components(model, learning_rate)


In [ ]:
# Training loop
def train_one_epoch(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(data_loader)
    print(f"Loss: {avg_loss:.4f}")


#📈 8. Evaluation and Model Saving

In [ ]:
# Evaluation function
def evaluate_model(model, validation_loader, device):
    model.eval()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for batch in validation_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            total_loss += loss.item()

            preds = torch.argmax(logits, dim=-1)
            correct = (preds == labels).sum().item()
            total_correct += correct
            total_samples += labels.numel()

    avg_loss = total_loss / len(validation_loader)
    accuracy = total_correct / total_samples * 100

    print(f"Validation Loss: {avg_loss:.4f}")
    print(f"Validation Accuracy: {accuracy:.2f}%")


In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    train_one_epoch(model, train_loader, optimizer, device)


    model.eval()
    with torch.no_grad():
        evaluate_model(model, val_loader, device)

# Save the model after training
save_model(model, 'trained_model')

Epoch 1/10
Loss: 0.9554
Validation Loss: 0.5147
Validation Accuracy: 89.06%
Epoch 2/10
Loss: 0.5151
Validation Loss: 0.4526
Validation Accuracy: 90.17%
Epoch 3/10
Loss: 0.4222
Validation Loss: 0.4404
Validation Accuracy: 90.40%
Epoch 4/10
Loss: 0.3596
Validation Loss: 0.4211
Validation Accuracy: 90.92%
Epoch 5/10
Loss: 0.3092
Validation Loss: 0.4186
Validation Accuracy: 91.03%
Epoch 6/10
Loss: 0.2685
Validation Loss: 0.4197
Validation Accuracy: 91.25%
Epoch 7/10
Loss: 0.2329
Validation Loss: 0.4337
Validation Accuracy: 91.24%
Epoch 8/10
Loss: 0.2023
Validation Loss: 0.4534
Validation Accuracy: 91.21%
Epoch 9/10
Loss: 0.1761
Validation Loss: 0.4706
Validation Accuracy: 91.25%
Epoch 10/10
Loss: 0.1538
Validation Loss: 0.4934
Validation Accuracy: 91.28%


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


# ☁️ 9. Push Model to Hugging Face Hub

In [ ]:
!pip install -U huggingface_hub

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from huggingface_hub import create_repo

repo_id = "Mostafa33/arabic-summarizer"
create_repo(repo_id, private=False)

RepoUrl('https://huggingface.co/Mostafa33/arabic-summarizer4', endpoint='https://huggingface.co', repo_type='model', repo_id='Mostafa33/arabic-summarizer4')

In [ ]:
from transformers import AutoTokenizer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('/content/trained_model')

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model.push_to_hub("arabic-summarizer")
tokenizer.push_to_hub("arabic-summarizer")


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


sentencepiece.bpe.model:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Mostafa33/arabic-summarizer4/commit/b91fefa34e3b0b3a9169115f8398318074dc3ba3', commit_message='Upload tokenizer', commit_description='', oid='b91fefa34e3b0b3a9169115f8398318074dc3ba3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Mostafa33/arabic-summarizer4', endpoint='https://huggingface.co', repo_type='model', repo_id='Mostafa33/arabic-summarizer4'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("Mostafa33/arabic-summarizer")
tokenizer = AutoTokenizer.from_pretrained("Mostafa33/arabic-summarizer")


# 10. Generate Summary from Paragraph

In [ ]:
def summarize_paragraph(paragraph, model, tokenizer, device, max_length=100):
    tokens = tokenizer.encode_plus(
        paragraph,
        max_length=512,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
    input_ids = tokens['input_ids'].to(device)
    attention_mask = tokens['attention_mask'].to(device)

    summary_tokens = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=max_length
    )

    summary = tokenizer.decode(summary_tokens[0], skip_special_tokens=True)
    return summary


In [ ]:
model = model.to(device)

In [ ]:
input_paragraph = """
في ظل التغيرات المتسارعة التي يشهدها العالم اليوم، تبرز أهمية التعليم كأداة رئيسية لتحقيق التنمية المستدامة وبناء المجتمعات القادرة على التكيف مع متطلبات المستقبل. فقد أصبح من الضروري تحديث المناهج التعليمية وتطوير أساليب التدريس بما يتناسب مع احتياجات العصر الرقمي، حيث لم يعد الحفظ والتلقين كافيين لإعداد أجيال قادرة على مواجهة التحديات المعقدة. ولهذا، تتجه الكثير من الدول نحو اعتماد نماذج تعليمية تركز على التفكير النقدي، وحل المشكلات، والابتكار، والتعلم التفاعلي الذي يدمج بين النظرية والتطبيق.
إضافة إلى ذلك، تواجه المنظومات التعليمية في العالم العربي تحديات متعددة تشمل نقص التمويل، وتفاوت مستوى الجودة بين المناطق، وضعف البنية التحتية في بعض الدول. ويعد الاستثمار في تدريب المعلمين وتوفير بيئة تعليمية محفزة من العوامل الأساسية لتحسين جودة التعليم. كما أن تعزيز دور التكنولوجيا وتوظيفها في العملية التعليمية بات أمرًا ضروريًا، لا سيما بعد التجربة التي فرضتها جائحة كوفيد-19، والتي أظهرت أهمية التعلم عن بعد والحاجة إلى بنية تحتية رقمية قوية.
ومن ناحية أخرى، تلعب اللغة العربية دورًا محوريًا في العملية التعليمية في العالم العربي، ليس فقط كلغة تدريس، بل أيضًا كوسيلة لتعزيز الهوية الثقافية والحفاظ على التراث الحضاري. ومع ذلك، يواجه تعليم اللغة العربية تحديات في طرق تدريسها، وغياب التحديث في مناهجها، مما يستدعي جهودًا بحثية وتربوية لتطوير طرق فعالة لتعليمها تتناسب مع المتغيرات الحديثة وتلبي احتياجات المتعلمين.
في الختام، فإن مستقبل التعليم في العالم العربي يتطلب رؤية شاملة، وتخطيطًا استراتيجيًا، وتعاونًا بين الحكومات والمؤسسات التعليمية والمجتمع المدني، من أجل بناء نظام تعليمي عصري يُمكّن الأفراد من الإبداع والتميّز، ويُسهم في نهضة المجتمعات واستقرارها
"""
generated_summary = summarize_paragraph(input_paragraph, model, tokenizer, device, max_length=80)
print("Generated Summary:", generated_summary)


Generated Summary: في ظل التغيرات المتسارعة التي يشهدها العالم اليوم، تبرز أهمية التعليم كأداة رئيسية لتحقيق التنمية المستدامة وبناء المجتمعات القادرة على التكيف مع متطلبات المستقبل. كما أن تعزيز دور التكنولوجيا وتوظيفها في العملية التعليمية بات أمراا ضرورياا، لا سيما بعد التجربة التي فرضتها جائحة كوفيد-19، والتي أظهرت أهمية التعلم عن بعد، والحاجة إلى بنية تحتية رقمية قوية. ومن ناحية أخرى، تلعب


# 📊 11. Generate and Evaluate Predictions: ROUGE and BLEU


*   **ROUGE (Recall-Oriented Understudy for Gisting Evaluation):**

    Measures the overlap between generated and reference summaries based on n-gram, word sequences, and word pairs.

    *   ROUGE-1: Unigram overlap
    *   ROUGE-2: Bigram overlap

    *   ROUGE-L: Longest common subsequence (captures fluency)


*  **BLEU (Bilingual Evaluation Understudy Score):**
    Originally for machine translation, this metric evaluates how closely the generated summary matches reference summaries using n-gram precision.

In [ ]:
def generate_predictions(model, data_loader, tokenizer, device):
    model.eval()
    predictions = []
    true_summaries = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            generated_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=100)
            pred_summaries = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

            true_summary = tokenizer.decode(labels[0], skip_special_tokens=True)

            predictions.append(pred_summaries)
            true_summaries.append(true_summary)

    return predictions, true_summaries


In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6db78048d8dd4e36102aef69bc97f11bc77c67d4297adc887bfa5f7b3b770366
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

def calculate_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    rouge_scores = {'rouge-1': 0, 'rouge-2': 0, 'rouge-l': 0}
    for pred, ref in zip(predictions, references):
        scores = scorer.score(ref, pred)
        rouge_scores['rouge-1'] += scores['rouge1'].fmeasure
        rouge_scores['rouge-2'] += scores['rouge2'].fmeasure
        rouge_scores['rouge-l'] += scores['rougeL'].fmeasure
    return {key: value / len(predictions) for key, value in rouge_scores.items()}


def calculate_bleu(predictions, references):
    bleu_scores = []
    for pred, ref in zip(predictions, references):
        pred_tokens = pred.split()
        ref_tokens = ref.split()
        bleu_scores.append(sentence_bleu([ref_tokens], pred_tokens))
    return sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0


In [ ]:
# Generate predictions for fine-tuned model
fine_tuned_predictions, fine_tuned_true_summaries = generate_predictions(model, val_loader, tokenizer, device)

# Calculate ROUGE and BLEU scores for fine-tuned model
rouge_scores = calculate_rouge(fine_tuned_predictions, fine_tuned_true_summaries)
bleu_score = calculate_bleu(fine_tuned_predictions, fine_tuned_true_summaries)

print("\n Evaluation - Fine-Tuned Model:")
print(f"ROUGE-1 Score: {rouge_scores['rouge-1']:.4f}")
print(f"ROUGE-2 Score: {rouge_scores['rouge-2']:.4f}")
print(f"ROUGE-L Score: {rouge_scores['rouge-l']:.4f}")
print(f"BLEU Score: {bleu_score:.4f}")



 Evaluation - Fine-Tuned Model:
ROUGE-1 Score: 0.0233
ROUGE-2 Score: 0.0014
ROUGE-L Score: 0.0221
BLEU Score: 0.2406


In [ ]:
# Generate predictions and references for the test set
test_preds, test_refs = generate_predictions(model, test_loader, tokenizer, device)

# Compute ROUGE and BLEU scores for the test set
test_rouge = calculate_rouge(test_preds, test_refs)
test_bleu = calculate_bleu(test_preds, test_refs)


print("\nTest Set Evaluation:")
print(f"ROUGE-1: {test_rouge['rouge-1']:.4f}")
print(f"ROUGE-2: {test_rouge['rouge-2']:.4f}")
print(f"ROUGE-L: {test_rouge['rouge-l']:.4f}")
print(f"BLEU Score: {test_bleu:.4f}")


Test Set Evaluation:
ROUGE-1: 0.0274
ROUGE-2: 0.0032
ROUGE-L: 0.0274
BLEU Score: 0.2031


# 12. Evaluate Pretrained AraBART for Comparison

In [ ]:
pretrained_model = AutoModelForSeq2SeqLM.from_pretrained('moussaKam/AraBART').to(device)
pretrained_tokenizer = AutoTokenizer.from_pretrained('moussaKam/AraBART')

# Step 5: Generate predictions from the pretrained model
pretrained_predictions, pretrained_true_summaries = generate_predictions(pretrained_model, val_loader, pretrained_tokenizer, device)

# Step 6: Evaluate pretrained model
pretrained_rouge_scores = calculate_rouge(pretrained_predictions, pretrained_true_summaries)
pretrained_bleu_score = calculate_bleu(pretrained_predictions, pretrained_true_summaries)

print("\nEvaluation - Pretrained AraBART:")
print(f"ROUGE-1 Score: {pretrained_rouge_scores['rouge-1']:.4f}")
print(f"ROUGE-2 Score: {pretrained_rouge_scores['rouge-2']:.4f}")
print(f"ROUGE-L Score: {pretrained_rouge_scores['rouge-l']:.4f}")
print(f"BLEU Score: {pretrained_bleu_score:.4f}")



Evaluation - Pretrained AraBART:
ROUGE-1 Score: 0.0015
ROUGE-2 Score: 0.0000
ROUGE-L Score: 0.0015
BLEU Score: 0.0728
